## 根据天凤的牌谱文件生成训练使用的dataset

In [ ]:
import os, sys, re, json, torch
from components.Taikyoku_loader import Taikyoku_loader
from components.encoder import Encoder
from enum import Enum
import numpy as np

def default(obj):
    if isinstance(obj, Enum):
        return obj.value
    raise TypeError(f"{obj!r} is not JSON serializable")

os.makedirs("../../dataset/Pretaining", exist_ok=True)
dataset_path = "../../dataset/Tenhou/2011_nan_4_kuitan_aka"
output_path = "../../dataset/Pretaining"

seq_buffer = []
masked_info_buffer = []
batch_index = 0

# 日志文件名的正则表达式，数字开头".log"结尾
file_name_pattern = re.compile(r"^\d+\.log$")
file_list = os.listdir(dataset_path)
file_list = [f for f in file_list if file_name_pattern.match(f)]

# 读取所有日志文件
for file in file_list:
    loader = Taikyoku_loader(os.path.join(dataset_path, file))

    # 从每场对局中随机提取3个局面
    for kyoku_index, kyoku in enumerate(loader.log):
        loader.reset(num=kyoku_index+1)
        # 从每局中随机提取3个局面
        random_index = np.random.randint(1, len(kyoku)-1, 4)
        print(random_index)
        for action_index, _ in enumerate(kyoku):
            loader.step_forward()
            if action_index in random_index:
                output = loader.export_info()
                encoder = Encoder(output)
                encoded_seq, masked_info = encoder.encode()
                seq_buffer.append(encoded_seq.T)
                masked_info_buffer.append(masked_info.T)

                # if len(seq_buffer) == 320:
                #     # 转换为tensor并保存(320, seq_len, dim)
                #     seq_tensor = torch.tensor(seq_buffer, dtype=torch.float32)
                #     batch_index += 1
                #     seq_buffer = []
                #     masked_info_buffer = []
        


In [ ]:
load_tensor = torch.load(os.path.join(output_path, f"seq_0.pt"))
print(load_tensor)